In [11]:
!pip install Faker
!pip install python-slugify

DEPRECATION: jupyter-server 2.0.0 has a non-standard dependency specifier jupyter-core!=~5.0,>=4.12. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of jupyter-server or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
DEPRECATION: jupyter-server 2.0.0 has a non-standard dependency specifier jupyter-core!=~5.0,>=4.12. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of jupyter-server or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11

In [12]:
import pandas as pd
from faker import Faker
import slugify

# number of rows
n = 1000

fake = Faker('fr_FR')

dest_path = './datasets/'
person_file = dest_path + 'persons'

df = pd.DataFrame(columns=['first_name', 'last_name', 'email', 'phone', 'national_number'])

def generate_person():
    return {
        'first_name': fake.first_name(),
        'last_name': fake.last_name(),
        'email': fake.email().replace('example', slugify.slugify(fake.company())),
        'phone': fake.phone_number().replace('+33', '+32'),
        # national number = format YY.MM.DD-XXX.XX
        'national_number': fake.date_of_birth(minimum_age=25, maximum_age=77).strftime('%y.%m.%d') + '-' + str(fake.random_number(digits=3)).zfill(3) + '.' + str(fake.random_number(digits=2)).zfill(2)
    }

for _ in range(n):
    new_person = generate_person()
   
    temp_df = pd.DataFrame(new_person, index=[0])
    df = pd.concat([df, temp_df], ignore_index=True)
    
df.to_excel(person_file + '.xlsx', index=False)
df.to_csv(person_file + '.csv', index=False)
df.to_html(person_file + '.html', index=False)
df.to_parquet(person_file + '.parquet', index=False)

#df.to_csv(person_file + '.csv.gz', index=False, compression='gzip')
#df.to_excel(person_file + '.xlsx', index=False)
#df.to_json(person_file + '.json', orient='records')

In [13]:
import random

# subset is a random percentage of the dataset
subset_n = random.randint(n // 4, n)

# random subset of national_number of the dataset
random_persons = df.sample(subset_n)
# only keep the national_number
haircuts = ['Bob cut', 'Bouffant', 'Bowl cut', 'Butch cut', 'Mullet', 'mULLET', 'Hi-top fade', 'Undercut', 'Tonsure', 'Pixie cut', 'Pompadour', 'Layered hair', 'Big hair', 'Bun', 'Chignon', 'Chonmage', 'Devilock', 'Dreadlocks' , 'devilock' ]
# add random haircut to the subset

random_persons['haircut'] = [random.choice(haircuts) for _ in range(subset_n)]

random_persons['white_hair_count'] = [random.randint(0, 500) for _ in range(subset_n)]

# define array of one true and 9000 false
true_or_false = [True] + [False] * random.randint(2, 9)

# randomily append a k at the end of the white_hair_count
random_persons['white_hair_count'] = random_persons['white_hair_count'].apply(lambda x: str(x) + 'k' if random.choice(true_or_false) else x)

# 10% of white_hair_count will be None
random_persons['white_hair_count'] = random_persons['white_hair_count'].apply(lambda x: None if random.choice(true_or_false) else x)

# remove all non digits characters from the national_number
random_persons['national_number'] = random_persons['national_number'].str.replace(r'\D', '', regex=True)

# only keep the national_number and haircut
random_persons = random_persons[['national_number', 'haircut', 'white_hair_count']]

# rename col national_number with NATIONAL_NUMBER_CAPS_LOVER
random_persons = random_persons.rename(columns={'national_number': 'NATIONAL_NUMBER_CAPS_LOVER'})

random_persons.to_csv(person_file + '_haircut.csv', index=False)

In [14]:
import sqlite3

conn = sqlite3.connect(person_file + '.db')
df.to_sql('persons', conn, index=False, if_exists='replace')
random_persons.to_sql('haircuts', conn, index=False, if_exists='replace')

721